# Import packages

In [1]:
import pandas as pd

In [2]:
import plotly
import datapane as dp
plotly.offline.init_notebook_mode(connected=True)

In [3]:
import random

In [4]:
import re
import pickle


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Upload json files

In [5]:
#files= ['0.json', '1.json', '2.json', '3.json', '4.json','42.json', '15693.json']
files = ['0.json']
# create an empty list to store the DataFrames
dfsList = [pd.read_json(file, orient='records') for file in files]

# combine all DataFrames into a single DataFrame
crossrefDF = pd.concat(dfsList, ignore_index=True)


In [6]:
crossrefDF

,items
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',..."
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...
2,{'abstract': '<jats:p> <jats:bold>...
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...
4,{'abstract': '<jats:p>The integration of the I...
...,...
4995,{'URL': 'http://dx.doi.org/10.1053/tvjl.2002.0...
4996,{'abstract': '<jats:p>This article reports on ...
4997,{'abstract': '<jats:p> We present a collection...
4998,{'abstract': '<jats:p>Looks at the nature of w...


# Data preparation 

In [7]:
noAuthors = [i for i in range(len(crossrefDF)) if 'author' not in crossrefDF['items'][i]]

Authors = [i for i in range(len(crossrefDF)) if 'author'  in crossrefDF['items'][i]]

In [8]:
len(noAuthors) + len(Authors) == len(crossrefDF)

True

## Rows with authors

In [9]:
crossrefAuth = crossrefDF.iloc[Authors].copy()

crossrefAuth.reset_index(inplace= True)
crossrefAuth.drop(columns = ['index'], inplace = True)


## Extract 'DOI'

In [10]:
crossrefAuth.loc[:, 'DOI'] = crossrefAuth['items'].apply(lambda x: x['DOI'])

In [11]:
crossrefAuth.head()

,items,DOI
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338


## Extract 'authors' --- number of authors

In [12]:
crossrefAuth.loc[:,'authors'] = crossrefAuth['items'].apply(lambda x: x['author'])


In [13]:
crossrefAuth.head()

,items,DOI,authors
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque..."
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904,"[{'given': 'S. V.', 'family': 'Akkelin', 'sequ..."
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se..."
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035,"[{'given': 'E.', 'family': 'Bakan', 'sequence'..."
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338,"[{'given': 'Kaba', 'family': 'Abdoulaye', 'seq..."


In [14]:
numAuthors = [len(crossrefAuth.iloc[i]['authors']) for i in range(len(crossrefAuth))]

In [15]:
## yparxoun lathi  ---> kalytera number of affiliations
crossrefAuth.loc[:,'# authors'] = numAuthors

In [16]:
crossrefAuth.head()

,items,DOI,authors,# authors
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904,"[{'given': 'S. V.', 'family': 'Akkelin', 'sequ...",3
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035,"[{'given': 'E.', 'family': 'Bakan', 'sequence'...",1
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338,"[{'given': 'Kaba', 'family': 'Abdoulaye', 'seq...",2


## Extract 'affiliations' --- number of affiliations

In [17]:
def getAff(k):
   return [crossrefAuth['authors'][k][j]['affiliation'] for j in range(len(crossrefAuth['authors'][k]))]
    

In [18]:
Affiliations = [getAff(k) for k in range(len(crossrefAuth))]

crossrefAuth.loc[:,'affiliations'] = Affiliations


In [19]:
numAffil = [len(Affiliations[i]) for i in range(len(crossrefAuth))]

In [20]:
crossrefAuth.loc[:,'# Affil'] = numAffil

In [21]:
crossrefAuth.head()

,items,DOI,authors,# authors,affiliations,# Affil
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904,"[{'given': 'S. V.', 'family': 'Akkelin', 'sequ...",3,"[[], [], []]",3
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035,"[{'given': 'E.', 'family': 'Bakan', 'sequence'...",1,[[]],1
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338,"[{'given': 'Kaba', 'family': 'Abdoulaye', 'seq...",2,"[[], []]",2


## Clean 'empty' affiliations

In [22]:
possibleEmptyAff = []

for k in range(len(crossrefAuth)):
    if len(crossrefAuth['affiliations'][k][0]) == 0:
        possibleEmptyAff.append(k)

In [23]:
len(possibleEmptyAff)

3767

In [24]:
crossrefAuth['affiliations'].iloc[2611]

[[],
 [],
 [],
 [],
 [{'name': 'Department of Organic and Polymeric Materials, Tokyo Institute of Technology, 2-12-1, Ohokayama, Meguro-ku, Tokyo 152-8552, Japan'}]]

In [25]:
nonEmptyAff = []

for k in possibleEmptyAff:
    for j in range(len(crossrefAuth['affiliations'].iloc[k])):
        if len(crossrefAuth['affiliations'].iloc[k][j]) != 0:
            nonEmptyAff.append(k)
    
    

In [26]:

FinalEmptyyAff=  [x for x in possibleEmptyAff if x not in nonEmptyAff] 

In [27]:
FinalNonEmptyAff = [x for x in range(len(crossrefAuth)) if x not in FinalEmptyyAff]

# affilDF: crossrefAuth subdataframe with nonpempty affiliation lists

In [28]:
affilDF = crossrefAuth.iloc[FinalNonEmptyAff].copy()
affilDF.reset_index(inplace = True)
affilDF.drop(columns = ['index'], inplace = True)

## (still some cleaning: cases with empty brackets [{}])

In [29]:
affilDF[affilDF['DOI'] == '10.48130/emst-2022-0020']

,items,DOI,authors,# authors,affiliations,# Affil


In [30]:
for k in range(len(affilDF)):
    if len(affilDF['affiliations'][k][0]) != 0 and affilDF['affiliations'][k][0][0] == {}:
        print(k)

In [31]:
emptyBrackets = [k for k in range(len(affilDF)) if len(affilDF['affiliations'][k][0]) != 0 and affilDF['affiliations'][k][0][0] == {}]

In [32]:
affilDF.iloc[emptyBrackets]

,items,DOI,authors,# authors,affiliations,# Affil


In [33]:
affilDF.copy()

,items,DOI,authors,# authors,affiliations,# Affil
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3
1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10
2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3
3,{'abstract': '<jats:p> The nuclear...,10.1126/science.1073633,"[{'given': 'Jianping', 'family': 'Hu', 'sequen...",6,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...",6
4,{'URL': 'http://dx.doi.org/10.1089/10872910260...,10.1089/10872910260066679,"[{'given': 'Edward E.', 'family': 'Eaton', 'se...",5,[[{'name': 'Albert Einstein College of Medicin...,5
...,...,...,...,...,...,...
882,"{'URL': 'http://dx.doi.org/10.1021/ic0255961',...",10.1021/ic0255961,"[{'given': 'Fatima', 'family': 'Setifi', 'sequ...",11,[[{'name': 'Laboratoire de Chimie du Solide et...,11
883,"{'URL': 'http://dx.doi.org/10.1021/jf011259j',...",10.1021/jf011259j,"[{'given': 'Alexandre G.', 'family': 'Torres',...",3,[[{'name': 'Laboratório de Bioquímica Nutricio...,3
884,"{'URL': 'http://dx.doi.org/10.1021/jp014423e',...",10.1021/jp014423e,"[{'given': 'Keisaku', 'family': 'Kimura', 'seq...",3,"[[{'name': 'Department of Material Science, Hi...",3
885,{'abstract': '<jats:p> The actin filament-asso...,10.1091/mbc.e01-12-0148,"[{'given': 'Yong', 'family': 'Qian', 'sequence...",12,[[{'name': 'The Mary Babb Randolph Cancer Cent...,12


In [34]:
affilDF.drop(emptyBrackets, inplace = True)

In [35]:
affilDF.reset_index(inplace = True)

In [36]:
affilDF.copy()

,index,items,DOI,authors,# authors,affiliations,# Affil
0,0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3
1,1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10
2,2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3
3,3,{'abstract': '<jats:p> The nuclear...,10.1126/science.1073633,"[{'given': 'Jianping', 'family': 'Hu', 'sequen...",6,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...",6
4,4,{'URL': 'http://dx.doi.org/10.1089/10872910260...,10.1089/10872910260066679,"[{'given': 'Edward E.', 'family': 'Eaton', 'se...",5,[[{'name': 'Albert Einstein College of Medicin...,5
...,...,...,...,...,...,...,...
882,882,"{'URL': 'http://dx.doi.org/10.1021/ic0255961',...",10.1021/ic0255961,"[{'given': 'Fatima', 'family': 'Setifi', 'sequ...",11,[[{'name': 'Laboratoire de Chimie du Solide et...,11
883,883,"{'URL': 'http://dx.doi.org/10.1021/jf011259j',...",10.1021/jf011259j,"[{'given': 'Alexandre G.', 'family': 'Torres',...",3,[[{'name': 'Laboratório de Bioquímica Nutricio...,3
884,884,"{'URL': 'http://dx.doi.org/10.1021/jp014423e',...",10.1021/jp014423e,"[{'given': 'Keisaku', 'family': 'Kimura', 'seq...",3,"[[{'name': 'Department of Material Science, Hi...",3
885,885,{'abstract': '<jats:p> The actin filament-asso...,10.1091/mbc.e01-12-0148,"[{'given': 'Yong', 'family': 'Qian', 'sequence...",12,[[{'name': 'The Mary Babb Randolph Cancer Cent...,12


In [37]:
affilDF.drop(columns = ['index'], inplace = True)

# Clean affiliations 

## is_contained(a,b) map : returns true when a is a substring of b 

In [38]:
def is_contained(s, w):
    words = s.split()  # Split the string 's' into a list of words
    for word in words:
        if word not in w:  # If a word from 's' is not found in 'w'
            return False  # Return False immediately
    return True  # If all words from 's' are found in 'w', return True

## 1. "Unique" affiliations --- number of unique affiliations

In [39]:
uniqueAff = [] 

for i in range(len(affilDF)):
    uniqueAff.append(list(set([x[0] for x in [list(d.values()) for d in  [item for sublist in affilDF['affiliations'].iloc[i] for item in sublist]]])))


In [40]:
affilDF.loc[:,'uniqueAff'] = uniqueAff

In [41]:
numUniqueAff = [len(affilDF['uniqueAff'].iloc[i]) for i in range(len(affilDF))]

In [42]:
affilDF.loc[:,'# uniqueAff'] = numUniqueAff

In [43]:
affilDF.head()

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3,"[Department of Chemistry, Texas A&M University...",1
1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10,"[From the Section of Cardiology (D.K.K., X.Z.,...",1
2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3,"[From the Department of Physiology, Medical Co...",1
3,{'abstract': '<jats:p> The nuclear...,10.1126/science.1073633,"[{'given': 'Jianping', 'family': 'Hu', 'sequen...",6,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...",6,"[Plant Biology Laboratory,, Laboratory of Neur...",3
4,{'URL': 'http://dx.doi.org/10.1089/10872910260...,10.1089/10872910260066679,"[{'given': 'Edward E.', 'family': 'Eaton', 'se...",5,[[{'name': 'Albert Einstein College of Medicin...,5,"[Albert Einstein College of Medicine, Bronx, N...",4


## 2. Remove stop words ['from', 'the']

In [44]:
stopWords = ['from', 'the', 'From', 'The']

In [45]:
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopWords]
    return ' '.join(filtered_words)


# apply the function to the column  affilDF['uniqueAff'] to create column affilDF.loc[:,'uniqueAff1']

affilDF.loc[:,'uniqueAff1'] = affilDF['uniqueAff'].apply(lambda x: [remove_stop_words(s) for s in x])


## 3. Remove parenthesis 

In [46]:
def remove_parentheses(text):
   return re.sub(r'\([^()]*\)', '', text)

# apply the function to each list element of column affilDF['uniqueAff1'] to remove substrings inside parentheses

affilDF.loc[:,'uniqueAff1'] = affilDF['uniqueAff1'].apply(lambda x: [remove_parentheses(s) for s in x])


## 4. Remove @#$%characters 

In [47]:
affNoSymbols = []

for i in range(len(list(affilDF['uniqueAff1']))):
    L = list(affilDF['uniqueAff1'])[i]
    for i in range(len(L)):
        L[i] = re.sub(r'[^\w\s,Α-Ωα-ωぁ-んァ-ン一-龯，]', '', L[i])
    affNoSymbols.append(L)


In [48]:
affNoSymbols = [[item for item in inner_list if item != "inc"] for inner_list in affNoSymbols]


In [49]:
affilDF['uniqueAff1'] = affNoSymbols


## 5. Check 'sub'-affiliations (affiliations that are contained in other affiliations of the same DOI)

In [50]:
newAff0 = []

for k in range(len(affilDF)):
    
    L2 = []
    for s1 in affilDF['uniqueAff1'].iloc[k]:
        is_substring = False
        for s2 in affilDF['uniqueAff1'].iloc[k]:
            if s1 != s2 and s1 in s2:
                is_substring = True
                break
        if not is_substring:
            L2.append(s1)
    newAff0.append(L2)

In [51]:
newAffList = [list(set(newAff0[k])) for k in range(len(newAff0))]

In [52]:
affilDF['uniqueAff2'] = newAffList

In [53]:
affilDF.head(3)

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff,uniqueAff1,uniqueAff2
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3,"[Department of Chemistry, Texas A&M University...",1,"[Department of Chemistry, Texas AM University,...","[Department of Chemistry, Texas AM University,..."
1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10,"[From the Section of Cardiology (D.K.K., X.Z.,...",1,[Section of Cardiology and Echocardiography L...,[Section of Cardiology and Echocardiography L...
2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3,"[From the Department of Physiology, Medical Co...",1,"[Department of Physiology, Medical College of ...","[Department of Physiology, Medical College of ..."


## 6. Split strings where ',' or ';' appears    | Apply .lower()

In [252]:
def substringsDict(string):
    
    split_strings = [re.sub(r'^[\s.]+|[\s.]+$', '', s.strip()) for s in re.split(r'[,;]', string)]
    dict_string = {i: split_strings[i] for i in range(len(split_strings)) if split_strings[i]}
    dict_string = {key: value.lower() for key, value in dict_string.items()}

    return dict_string


In [253]:
newAffkomma = []

for k in range(len(affilDF)):
    
    new_list = []
    for item in affilDF['uniqueAff2'].iloc[k]:
        new_list.append(substringsDict(item))

    newAffkomma.append(new_list)



In [254]:
affilDF['uniqueAff4'] = [list(x[0].values()) for x in newAffkomma]

In [255]:
removeList = ['university','research institute','laboratory' , 'universit','gmbh', 'inc', 'university of', 'research center', 
'university college','national institute of', 'school of medicine', "university school", 'graduate school of', 'graduate school of engineering', 
'institute of tropical medicine', 'institute of virology', 'faculty of medicine','laboratory', 'university park', 'institute of science']

# Labels based on legalnames of openAIRE's organizations

In [256]:
uniList = ['institu', 'istituto','univ', 'college', 'center', 'centre' , 'cnrs', 'faculty','school' , 'academy' , 'école', 'hochschule' , 'ecole' ]

labList = ['lab']

hosplList = ['hospital' ,'clinic', 'hôpital']

gmbhList = ['gmbh', 'company' , 'industr', 'etaireia' , 'corporation', 'inc']

musList =  ['museum', 'library']

foundList =  ['foundation' , 'association','organization' ,'society', 'group' ]

deptList = ['district' , 'federation'  , 'government' , 'municipal' , 'county', 'ministry','council', 'agency']

unknownList = ['unknown']

#######   Dictionaries ##########

uniDict = {k: 'Univ/Inst' for k in uniList}   

labDict = {k: 'Laboratory' for k in labList} 

hosplDict = {k: 'Hospital' for k in hosplList}   

gmbhDict = {k: 'Company' for k in gmbhList}   

musDict = {k: 'Museum' for k in musList}   

#schoolDict = {k: 'School' for k in schoolList}   

foundDict = {k: 'Foundation' for k in foundList}   

deptDict = {k: 'Government' for k in deptList}   

unknownDict =  {k: 'Unknown' for k in unknownList}   

categDictsList = [uniDict, labDict, hosplDict, gmbhDict, musDict, #schoolDict, 
                  foundDict, deptDict, unknownDict]

################# Final Dictionary #####################

categDicts = {}
i = 0
while i in range(len(categDictsList)):
    categDicts.update(categDictsList[i])
    i = i+1
    
    


## affiliationsDict

In [257]:
affiliationsDict = {}

for i in range(len(affilDF)):
    affiliationsDict[i] = affilDF['uniqueAff4'].iloc[i]
    


In [258]:
d_new = {}

# iterate over the keys of affiliationsDict
for k in range(len(affiliationsDict)):
    # get the list associated with the current key in affiliationsDict
    L = affiliationsDict.get(k, [])
    
    # apply the mapping using a list comprehension
    mapped_list = [[s, v] for s in L for k2, v in categDicts.items() if k2 in s]
    
    # add the mapped list to the new dictionary d_new
    d_new[k] = mapped_list

In [259]:
affilDF['Dictionary'] = d_new

## What is not included 

In [260]:
notInList = [i for i in range(len(affilDF)) if len(affilDF['Dictionary'].iloc[i]) == 0]
    

In [261]:
affilDF.iloc[notInList]

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff,uniqueAff1,uniqueAff2,uniqueAff4,Dictionary
5,{'URL': 'http://dx.doi.org/10.1126/science.296...,10.1126/science.296.5576.2123a,"[{'given': 'Wayne', 'family': 'Kondro', 'seque...",1,[[{'name': 'Wayne Kondro writes from Ottawa.'}]],1,[Wayne Kondro writes from Ottawa.],1,[Wayne Kondro writes Ottawa],[Wayne Kondro writes Ottawa],[wayne kondro writes ottawa],[]
6,{'URL': 'http://dx.doi.org/10.1046/j.1440-1665...,10.1046/j.1440-1665.2002.t01-1-00428.x,"[{'given': 'Bob', 'family': 'Adler', 'sequence...",1,"[[{'name': 'Melbourne, Victoria'}]]",1,"[Melbourne, Victoria]",1,"[Melbourne, Victoria]","[Melbourne, Victoria]","[melbourne, victoria]",[]
7,{'abstract': '<jats:p>Nonresonant manipulation...,10.1126/science.1072245,"[{'given': 'Thilo M.', 'family': 'Brill', 'seq...",8,"[[{'name': 'Schlumberger-Doll Research, 36 Old...",8,"[Schlumberger-Doll Research, 36 Old Quarry Roa...",1,"[SchlumbergerDoll Research, 36 Old Quarry Road...","[SchlumbergerDoll Research, 36 Old Quarry Road...","[schlumbergerdoll research, 36 old quarry road...",[]
11,{'URL': 'http://dx.doi.org/10.1161/01.cir.0000...,10.1161/01.cir.0000019680.58760.ac,"[{'given': 'Ruth', 'family': 'SoRelle', 'seque...",1,[[{'name': 'Circulation Newswriter'}]],1,[Circulation Newswriter],1,[Circulation Newswriter],[Circulation Newswriter],[circulation newswriter],[]
23,"{'URL': 'http://dx.doi.org/10.1021/op0255130',...",10.1021/op0255130,"[{'given': 'Bryan H.', 'family': 'Landis', 'se...",4,[[{'name': 'Bioprocess and Formulation Technol...,4,"[Bioprocess and Formulation Technology, Pharma...",1,"[Bioprocess and Formulation Technology, Pharma...","[Bioprocess and Formulation Technology, Pharma...","[bioprocess and formulation technology, pharma...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
794,"{'URL': 'http://dx.doi.org/10.1149/1.1391937',...",10.1149/1.1391937,"[{'given': 'Patricia A.', 'family': 'O'Neil', ...",5,[[{'name': 'Department of Electrical and Compu...,5,[Department of Electrical and Computer Enginee...,2,[Department of Electrical and Computer Enginee...,[Department of Electrical and Computer Enginee...,[department of electrical and computer enginee...,[]
802,{'URL': 'http://dx.doi.org/10.1161/01.cir.0000...,10.1161/01.cir.0000013182.44212.7a,"[{'given': 'Paolo', 'family': 'Ferrari', 'sequ...",1,[[{'name': 'Division of Nephrology and Hyperte...,1,"[Division of Nephrology and Hypertension, Inse...",1,"[Division of Nephrology and Hypertension, Inse...","[Division of Nephrology and Hypertension, Inse...","[division of nephrology and hypertension, inse...",[]
816,{'abstract': '<jats:p> <jats:bold>...,10.1161/hs0202.102950,"[{'given': 'John R.', 'family': 'Toomey', 'seq...",9,[[{'name': 'From the Departments of Cardiovasc...,9,[From the Departments of Cardiovascular Pharma...,1,"[Departments of Cardiovascular Pharmacology , ...","[Departments of Cardiovascular Pharmacology , ...","[departments of cardiovascular pharmacology, n...",[]
836,{'URL': 'http://dx.doi.org/10.1105/tpc.3.3.259...,10.1105/tpc.3.3.259,"[{'given': 'J F', 'family': 'Manen', 'sequence...",6,"[[{'name': 'Jardin Botanique, Genève, Switzerl...",6,"[Jardin Botanique, Genève, Switzerland.]",1,"[Jardin Botanique, Genève, Switzerland]","[Jardin Botanique, Genève, Switzerland]","[jardin botanique, genève, switzerland]",[]


In [262]:
len(affilDF)  - len(notInList)

820

In [263]:
len(notInList)

67

# affilDF1 ['DOI', 'affiliations', 'Dictionary','uniqueAff4', 'uniqueAff2','# authors','# uniqueAff']

In [264]:
affilDF1 = affilDF[['DOI', 'affiliations', 'Dictionary','uniqueAff4', 'uniqueAff2','# authors','# uniqueAff']]

## New column: category based on the labels 

In [265]:
category = [', '.join(list(set([x[1] for x in affilDF1['Dictionary'].iloc[i]]))) for i in range(len(affilDF1))]
    


In [266]:
affilDF1 = affilDF1.copy()

In [267]:
affilDF1.loc[:, 'category'] = category


### new label: rest

In [268]:
for i in range(len(affilDF1)):
    if len(affilDF1['category'].iloc[i]) == 0:
        affilDF1.iloc[i, affilDF1.columns.get_loc('category')] = 'Rest'


In [269]:
affiliationsSimple = [list(set([x[0] for x in affilDF1['Dictionary'].iloc[i]])) for i in range(len(affilDF1))]

In [270]:
city_names = ["Aberdeen", "Abilene", "Akron", "Albany", "Albuquerque", "Alexandria", "Allentown", "Amarillo", "Anaheim", "Anchorage", "Ann Arbor", "Antioch", "Apple Valley", "Appleton", "Arlington", "Arvada", "Asheville", "Athens", "Atlanta", "Atlantic City", "Augusta", "Aurora", "Austin", "Bakersfield", "Baltimore", "Barnstable", "Baton Rouge", "Beaumont", "Bel Air", "Bellevue", "Berkeley", "Bethlehem", "Billings", "Birmingham", "Bloomington", "Boise", "Boise City", "Bonita Springs", "Boston", "Boulder", "Bradenton", "Bremerton", "Bridgeport", "Brighton", "Brownsville", "Bryan", "Buffalo", "Burbank", "Burlington", "Cambridge", "Canton", "Cape Coral", "Carrollton", "Cary", "Cathedral City", "Cedar Rapids", "Champaign", "Chandler", "Charleston", "Charlotte", "Chattanooga", "Chesapeake", "Chicago", "Chula Vista", "Cincinnati", "Clarke County", "Clarksville", "Clearwater", "Cleveland", "College Station", "Colorado Springs", "Columbia", "Columbus", "Concord", "Coral Springs", "Corona", "Corpus Christi", "Costa Mesa", "Dallas", "Daly City", "Danbury", "Davenport", "Davidson County", "Dayton", "Daytona Beach", "Deltona", "Denton", "Denver", "Des Moines", "Detroit", "Downey", "Duluth", "Durham", "El Monte", "El Paso", "Elizabeth", "Elk Grove", "Elkhart", "Erie", "Escondido", "Eugene", "Evansville", "Fairfield", "Fargo", "Fayetteville", "Fitchburg", "Flint", "Fontana", "Fort Collins", "Fort Lauderdale", "Fort Smith", "Fort Walton Beach", "Fort Wayne", "Fort Worth", "Frederick", "Fremont", "Fresno", "Fullerton", "Gainesville", "Garden Grove", "Garland", "Gastonia", "Gilbert", "Glendale", "Grand Prairie", "Grand Rapids", "Grayslake", "Green Bay", "GreenBay", "Greensboro", "Greenville", "Gulfport-Biloxi", "Hagerstown", "Hampton", "Harlingen", "Harrisburg", "Hartford", "Havre de Grace", "Hayward", "Hemet", "Henderson", "Hesperia", "Hialeah", "Hickory", "High Point", "Hollywood", "Honolulu", "Houma", "Houston", "Howell", "Huntington", "Huntington Beach", "Huntsville", "Independence", "Indianapolis", "Inglewood", "Irvine", "Irving", "Jackson", "Jacksonville", "Jefferson", "Jersey City", "Johnson City", "Joliet", "Kailua", "Kalamazoo", "Kaneohe", "Kansas City", "Kennewick", "Kenosha", "Killeen", "Kissimmee", "Knoxville", "Lacey", "Lafayette", "Lake Charles", "Lakeland", "Lakewood", "Lancaster", "Lansing", "Laredo", "Las Cruces", "Las Vegas", "Layton", "Leominster", "Lewisville", "Lexington", "Lincoln", "Little Rock", "Long Beach", "Lorain", "Los Angeles", "Louisville", "Lowell", "Lubbock", "Macon", "Madison", "Manchester", "Marina", "Marysville", "McAllen", "McHenry", "Medford", "Melbourne", "Memphis", "Merced", "Mesa", "Mesquite", "Miami", "Milwaukee", "Minneapolis", "Miramar", "Mission Viejo", "Mobile", "Modesto", "Monroe", "Monterey", "Montgomery", "Moreno Valley", "Murfreesboro", "Murrieta", "Muskegon", "Myrtle Beach", "Naperville", "Naples", "Nashua", "Nashville", "New Bedford", "New Haven", "New London", "New Orleans", "New York", "New York City", "Newark", "Newburgh", "Newport News", "Norfolk", "Normal", "Norman", "North Charleston", "North Las Vegas", "North Port", "Norwalk", "Norwich", "Oakland", "Ocala", "Oceanside", "Odessa", "Ogden", "Oklahoma City", "Olathe", "Olympia", "Omaha", "Ontario", "Orange", "Orem", "Orlando", "Overland Park", "Oxnard", "Palm Bay", "Palm Springs", "Palmdale", "Panama City", "Pasadena", "Paterson", "Pembroke Pines", "Pensacola", "Peoria", "Philadelphia", "Phoenix", "Pittsburgh", "Plano", "Pomona", "Pompano Beach", "Port Arthur", "Port Orange", "Port Saint Lucie", "Port St. Lucie", "Portland", "Portsmouth", "Poughkeepsie", "Providence", "Provo", "Pueblo", "Punta Gorda", "Racine", "Raleigh", "Rancho Cucamonga", "Reading", "Redding", "Reno", "Richland", "Richmond", "Richmond County", "Riverside", "Roanoke", "Rochester", "Rockford", "Roseville", "Round Lake Beach", "Sacramento", "Saginaw", "Saint Louis", "Saint Paul", "Saint Petersburg", "Salem", "Salinas", "Salt Lake City", "San Antonio", "San Bernardino", "San Buenaventura", "San Diego", "San Francisco", "San Jose", "Santa Ana", "Santa Barbara", "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Maria", "Santa Rosa", "Sarasota", "Savannah", "Scottsdale", "Scranton", "Seaside", "Seattle", "Sebastian", "Shreveport", "Simi Valley", "Sioux City", "Sioux Falls", "South Bend", "South Lyon", "Spartanburg", "Spokane", "Springdale", "Springfield", "St. Louis", "St. Paul", "St. Petersburg", "Stamford", "Sterling Heights", "Stockton", "Sunnyvale", "Syracuse", "Tacoma", "Tallahassee", "Tampa", "Temecula", "Tempe", "Thornton", "Thousand Oaks", "Toledo", "Topeka", "Torrance", "Trenton", "Tucson", "Tulsa", "Tuscaloosa", "Tyler", "Utica", "Vallejo", "Vancouver", "Vero Beach", "Victorville", "Virginia Beach", "Visalia", "Waco", "Warren", "Washington", "Waterbury", "Waterloo", "West Covina", "West Valley City", "Westminster", "Wichita", "Wilmington", "Winston", "Winter Haven", "Worcester", "Yakima", "Yonkers", "York", "Youngstown"]

city_names = [x.lower() for x in city_names]

In [271]:
new_affiliations = []
for x in affiliationsSimple:
    new_x = []
    for elem in x:
        if elem not in city_names:
            new_x.append(elem)
    new_affiliations.append(new_x)



In [272]:
affilDF1.loc[:,'affilSimple'] = new_affiliations

In [273]:
affilDF1.head()

,DOI,affiliations,Dictionary,uniqueAff4,uniqueAff2,# authors,# uniqueAff,category,affilSimple
0,10.1021/ac020069k,"[[{'name': 'Department of Chemistry, Texas A&M...","[[texas am university, Univ/Inst], [college st...","[department of chemistry, texas am university,...","[Department of Chemistry, Texas AM University,...",3,1,Univ/Inst,[texas am university]
1,10.1161/01.cir.0000013846.72805.7e,[[{'name': 'From the Section of Cardiology (D....,[[section of cardiology and echocardiography ...,[section of cardiology and echocardiography l...,[Section of Cardiology and Echocardiography L...,10,1,"Laboratory, Univ/Inst","[baylor college of medicine, section of cardio..."
2,10.1161/hy02t2.102992,"[[{'name': 'From the Department of Physiology,...","[[medical college of wisconsin, Univ/Inst], [t...","[department of physiology, medical college of ...","[Department of Physiology, Medical College of ...",3,1,Univ/Inst,"[medical college of wisconsin, huazhong univer..."
3,10.1126/science.1073633,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...","[[howard hughes medical institute, Univ/Inst]]",[howard hughes medical institute],"[Howard Hughes Medical Institute,, Plant Biolo...",6,3,Univ/Inst,[howard hughes medical institute]
4,10.1089/10872910260066679,[[{'name': 'Albert Einstein College of Medicin...,"[[albert einstein college of medicine, Univ/In...","[albert einstein college of medicine, bronx, n...","[Albert Einstein College of Medicine, Bronx, N...",5,4,Univ/Inst,"[new york and jacobi medical center, albert ei..."


In [274]:
len(affilDF1[affilDF1['category'] == 'Rest'])

67

# UNIVS & LABS

In [275]:
univLabs = [i for i in range(len(affilDF1)) if 'Laboratory' in affilDF1['category'].iloc[i] 
            or 'Univ/Inst' in  affilDF1['category'].iloc[i]]

In [276]:

univLabsDF = affilDF1.iloc[univLabs].copy()

In [277]:
univLabsDF.reset_index(inplace = True)

In [278]:
univLabsDF.drop(columns = ['index'], inplace = True)

# Load files from openAIRE

In [279]:
with open('dixOpenAIRE_Alletc.pkl', 'rb') as f:
    dixOpenAIRE_Alletc = pickle.load(f)


In [280]:
with open('dixOpenAIRE_id.pkl', 'rb') as f:
    dixOpenAIRE_id = pickle.load(f)


In [281]:
with open('dixOpenOrgId.pkl', 'rb') as f:
    dixOpenOrgId = pickle.load(f)


## Clean/modify the files

In [282]:
dixOpenAIRE_Alletc1 =  {k.replace(',', ''): v for k, v in dixOpenAIRE_Alletc.items()}

In [283]:
dixOpenAIRE_id1 = {k.replace(',', ''): v for k, v in dixOpenAIRE_id.items()}

In [284]:
dixOpenOrgId1 = {k.replace(',', ''): v for k, v in dixOpenOrgId.items()}

In [285]:
def filter_key(key):
    # Remove all non-alphanumeric characters except Greek letters and Chinese characters
    return re.sub(r'[^\w\s,Α-Ωα-ωぁ-んァ-ン一-龯，]', '', key)

def filter_dictionary_keys(dictionary):
    filtered_dict = {}
    for key, value in dictionary.items():
        filtered_key = filter_key(key)
        filtered_dict[filtered_key] = value
    return filtered_dict


dixOpenAIRE_Alletc1 = filter_dictionary_keys(dixOpenAIRE_Alletc1)
dixOpenAIRE_id1 = filter_dictionary_keys(dixOpenAIRE_id1)
dixOpenOrgId1 = filter_dictionary_keys(dixOpenOrgId1)



In [286]:
del dixOpenAIRE_Alletc1['laboratory']
del dixOpenAIRE_Alletc1['university hospital']

In [287]:
del dixOpenAIRE_id1['laboratory']
del dixOpenAIRE_id1['university hospital']

In [288]:
del dixOpenOrgId1['university hospital']

# MATCHINGS

## Helper functions

 Correct the matchings

In [289]:
def bestSimScore(l1, l2, l3):
    """
    Finds the best match between a 'key word' and several legal names from the OpenAIRE database.
    ---> corrects special cases in the main map that follows

    Args:
        l1: List of unique affiliations.
        l2: List of matched OpenAIRE names.
        l3: List of pairs.

    Returns:
        List: Resulting list containing OpenAIRE names and their similarity scores.
    """
    
    vectorizer = CountVectorizer()
    numUniv = sum([(l1[i].lower()).count('univ') for i in range(len(l1))])
    result = []
    for i in range(len(l1)):
        best = [] 
        s = l1[i]
       # s_contains_university = is_contained("university", s.lower())  
        
        if not is_contained("univ", s.lower()):
            continue  # Skip if s does not contain "university" or "univ"
        
    
        for j in range(len(l2)):
            x = l2[j]  
              
            if l3[j][2] >=0.99 and (is_contained("univ", x.lower()) or is_contained("college", x.lower()) or  is_contained("center", x.lower())): # If the similarity score of a pair (s,x) was 1, we store it to results list
                result.append([l2[j], 1])
                
            else:
        #        x_contains_university = is_contained("university", x.lower())
                if not is_contained("univ", x.lower()):
                    continue  # Skip if x does not contain "university" or "univ"
                
            
                x_vector = vectorizer.fit_transform([x]).toarray() #Else we compute the similarity of s with the original affiiation name
                s_vector = vectorizer.transform([s]).toarray()

                # Compute similarity between the vectors
                similarity = cosine_similarity(x_vector, s_vector)[0][0]

                best.append([l2[j], similarity])
        
        if best:
            max_score = max(best, key=lambda x: x[1])[1]
            max_results = [(x[0], x[1]) for x in best if x[1] == max_score]
            if len(max_results) > 1:
                max_results.sort(key=lambda x: (l2.index(x[0]), -x[1]), reverse=False)
                result.append(max_results[-1])
            else:
                result.append(max_results[0])
                
    univ_list = []
    other_list = []
    
    for r in result:
        if is_contained('univ',r[0]):
            univ_list.append(r)
        else:
            other_list.append(r)
            
    if len(univ_list)> numUniv:
        result = univ_list[:numUniv] + other_list
                
    return result

### find rows with multiple mathcings

In [291]:
def index_multipleMatchings(df):
    multipleMatchings = []

    for i in range(len(df)):
        result_dict = {}

        for t in [t[0] for t in df.Pairs.iloc[i]]:
            key = t
            if key in result_dict:
                result_dict[key] += 1
                multipleMatchings.append(i)
                
            else:
                result_dict[key] = 1
    return list(set(multipleMatchings))
                
        

## Main map

In [292]:
def Doi_Ids(m, DF, dixOpenAIRE, simU, simG):
    
    """
    Matches affiliations in DataFrame 'DF' with names from dictionary 'dixOpenAIRE' and their openAIRE ids based on similarity scores.

    Args:
        m (int): The number of DOIs to check.
        DF (DataFrame): The input DataFrame containing affiliation data.
        dixOpenAIRE (dict): A dictionary of names from OpenAIRE.
        simU (float): Similarity threshold for universities.
        simG (float): Similarity threshold for non-universities.

    Returns:
        DataFrame: The final DataFrame with matched affiliations and their corresponding similarity scores.
    """
    
    lnamelist = list(dixOpenAIRE.keys())
    dix = {}    # will store indeces and legalnames of organizations of the DOI { i : [legalname1, legalname2,...]}
    deiktes = []  # stores indeces where a match is found
    vectorizer = CountVectorizer()
    similarity_ab = [] # stores lists of similarity scores of the mathces 
    pairs = [] #  pairs[i] =  [ [s,x,t] ] where (s,x) is a match and t the corresponding similarity score
    
    for k in range(m):
        similar_k = []
        pairs_k = []
        numMathcings_k =[]


        for s in DF['affilSimple'].iloc[k]:
            count_s = 0

            if s in lnamelist:
                deiktes.append(k)
                similarity = 1
                similar_k.append(similarity)
                
                pairs_k.append((s,s,similarity))
                count_s += 1

                if k not in dix:
                    dix[k] = [s]
                else:
                    dix[k].append(s)
            else:

                for x in lnamelist:
                    
                    if  is_contained(s, x):
                        x_vector = vectorizer.fit_transform([x]).toarray()
                        s_vector = vectorizer.transform([s]).toarray()

                        # Compute similarity between the vectors
                        similarity = cosine_similarity(x_vector, s_vector)[0][0]
                        if similarity > min(simU, simG):
                            if (is_contained('univ', s) and is_contained('univ', x)) and similarity > simU:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))
                                count_s = count_s+1


                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                            elif (not is_contained('univ', s) and not is_contained('univ', x)) and similarity > simG:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))
                                count_s += 1


                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                    elif is_contained(x, s):
                        if (is_contained('univ', s) and is_contained('univ', x)):

                            if 'and' in s:
                                list_s = s.split(' and ')
                                for t in list_s:
                                    if is_contained(x, t) and is_contained('univ', t):
                                        t_vector = vectorizer.fit_transform([t]).toarray()
                                        x_vector = vectorizer.transform([x]).toarray()

                            # Compute similarity between the vectors
                                        similarity = cosine_similarity(t_vector, x_vector)[0][0]
                                        if similarity > simU:
                                            similar_k.append(similarity)
                                            deiktes.append(k)
                                            pairs_k.append((s,x,similarity))
                                            count_s += 1



                                            if k not in dix:
                                                dix[k] = [x]
                                            else:
                                                dix[k].append(x)
                            
                            else: 
                                s_vector = vectorizer.fit_transform([s]).toarray()
                                x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                similarity = cosine_similarity(s_vector, x_vector)[0][0]
                                if similarity > 0.81: #max(0.82,sim):
                                    similar_k.append(similarity)
                                    deiktes.append(k)
                                    pairs_k.append((s,x,similarity))
                                    count_s += 1 



                                    if k not in dix:
                                        dix[k] = [x]
                                    else:
                                        dix[k].append(x)
                        elif not is_contained('univ', s) and not is_contained('univ', x):
                            if 'and' in s:
                                list_s = s.split(' and ')
                                for t in list_s:
                                    if is_contained(x, t):
                                        t_vector = vectorizer.fit_transform([t]).toarray()
                                        x_vector = vectorizer.transform([x]).toarray()

                            # Compute similarity between the vectors
                                        similarity = cosine_similarity(t_vector, x_vector)[0][0]
                                        if similarity > simG:
                                            similar_k.append(similarity)
                                            deiktes.append(k)
                                            pairs_k.append((s,x,similarity))
                                            count_s +=1



                                            if k not in dix:
                                                dix[k] = [x]
                                            else:
                                                dix[k].append(x)
                            
                            else: 
                                s_vector = vectorizer.fit_transform([s]).toarray()
                                x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                similarity = cosine_similarity(s_vector, x_vector)[0][0]
                                if similarity > simG: #max(0.82,sim):
                                    similar_k.append(similarity)
                                    deiktes.append(k)
                                    pairs_k.append((s,x,similarity))
                                    count_s +=1

                                    if k not in dix:
                                        dix[k] = [x]
                                    else:
                                        dix[k].append(x)
                            
        similarity_ab.append(similar_k)   
        similarity_ab = [lst for lst in similarity_ab if lst != []]
        pairs.append(pairs_k)
        
    perc = 100*len(dix)/m
    
    dixDoiAff = {DF['DOI'].iloc[key]: value for key, value in dix.items()} # dictionary {DOI : legalnames} 
    
    #dixDoiPid1 = {key : [dixOpenAIRE[x] for x in value if x in  lnamelist] for key , value in dixDoiAff.items()}
    
   # dixDoiPid = {key : [dixOpenAIRE[x] for x in value] for key , value in dixDoiAff.items()} # dictionary {DOI : PIDs} 
    
    
    
## Define the new Dataframe
    
    doiIdDF = pd.DataFrame()
    doiIdDF['DOI'] = list(dixDoiAff.keys())
    doiIdDF['Affiliations'] = list(DF['affiliations'].iloc[list(set(deiktes))])

    doiIdDF['Unique affiliations'] = list(DF['uniqueAff2'].iloc[list(set(deiktes))])
    
    doiIdDF['# Authors'] = list(DF['# authors'].iloc[list(set(deiktes))])


    doiIdDF['# Unique affiliations'] = list(DF['# uniqueAff'].iloc[list(set(deiktes))])

    doiIdDF['Candidates for matching'] = list(DF['affilSimple'].iloc[list(set(deiktes))])

    doiIdDF['Matched openAIRE names'] = list(dix.values())
    doiIdDF['# Matched orgs'] = [len(list(dix.values())[i]) for i in range(len(list(dix.values())))]
    

    doiIdDF['Similarity score'] = similarity_ab
    perfectSim = [[1 if num >= 1 else 0 for num in inner_list] for inner_list in similarity_ab]

    doiIdDF['Perfect match'] = perfectSim
    perfectSimSum = [sum(x) for x in perfectSim]
    doiIdDF['Perfect sum'] = perfectSimSum
    Pairs = [lst for lst in pairs if lst]
    doiIdDF['Pairs'] = Pairs
    


## Correct the matchings
    
    needCheck = [i for i  in range(len(doiIdDF)) if doiIdDF['# Matched orgs'].iloc[i] - max(doiIdDF['# Authors'].iloc[i],doiIdDF['# Unique affiliations'].iloc[i]) >0 or    i in index_multipleMatchings(doiIdDF)]

    ready = [i for i in range(len(doiIdDF)) if i not in needCheck]
   
    best = [ bestSimScore(doiIdDF['Unique affiliations'].iloc[i], doiIdDF['Matched openAIRE names'].iloc[i], doiIdDF['Pairs'].iloc[i]) for i in needCheck]
    best_o = []
    best_s = []
    
    for x in best:
        best_o.append([x[i][0]  for i in range(len(x))])
        best_s.append([round(x[i][1],2)  for i in range(len(x))])
    numMathced = [len(best_s[i]) for i in range(len(needCheck))]
    

    
    dfFinal0 = (doiIdDF.iloc[ready]).copy()
    dfFinal0['index'] = ready
    
    dfFinal1 = (doiIdDF.iloc[needCheck]).copy()
    dfFinal1['index'] = needCheck
    dfFinal1['Matched openAIRE names'] = best_o
    dfFinal1['Similarity score'] = best_s
    dfFinal1['# Matched orgs'] = numMathced
    
    finalDF =  pd.concat([dfFinal0, dfFinal1])
    finalDF.set_index('index', inplace=True)
    finalDF.sort_values('index', ascending=True, inplace = True)
    
    ids = [[dixOpenAIRE[x] for x in v] for v in finalDF['Matched openAIRE names']]
    numIds = [len(x) for x in ids]

    finalDF['IDs'] = ids
    finalDF['# IDs'] = numIds
# Assuming you want to remove rows where the 'column_name' meets a specific condition
    finalDF = finalDF[~(finalDF['# Matched orgs'] == 0)]
    finalDF = finalDF.reset_index(drop=True)


    
    
    return [perc, finalDF, doiIdDF]
    


In [293]:
result = Doi_Ids(len(univLabsDF), univLabsDF, dixOpenOrgId1, 0.7,0.82)

# HTML

In [294]:
mydf = result[1]
myDF = mydf[["DOI", "Unique affiliations",'# Authors','# Unique affiliations', '# Matched orgs','Candidates for matching', 'Matched openAIRE names', 'Similarity score']]


In [295]:
def update_Z(row):
    new_Z = []
    for i in range(len(row['IDs'])):
        entry = {'openaireId': row['IDs'][i], 'confidence': row['Similarity score'][i]}
        new_Z.append(entry)
    return new_Z

# Update the values in column 'Z' using 'apply'
mydf['affiliations'] = mydf.apply(update_Z, axis=1)


In [296]:
myDF_output = mydf[['DOI','affiliations']]
myDF_output = myDF_output.rename(columns={'DOI': 'doi'}).copy()



In [297]:
len(mydf)

613

In [298]:
app = dp.App(dp.Page(title="Matchings [explained]", blocks= [dp.Text('Matched '+str(result[0])+'%'), dp.Text('DOIs with aff. in Univ/Inst, Laboratories'), dp.Table(myDF)]
             ), 
           #  dp.Page(title="Final matchings", blocks= [dp.Text('Matched '+str(result[0])+'%'), dp.Text('DOIs with aff. in Univ/Inst, Laboratories'), dp.Table(myDF_output)]
             #)
             )
    


   
app.save(path="test.html", open=True)

App saved to ./test.html

# JSON

In [299]:
json0 = myDF_output.to_json(orient='records')

# Save the JSON to a file
with open('json0.json', 'w') as f:
    f.write(json0)